In [1]:
import torch
import torch.nn as nn
from torch_geometric.datasets import UPFD
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected
import numpy as np
from sklearn.metrics import f1_score
from models import GATClassifier,GNNClassifier
from utils import train_loop, test_loop

train_dataset = UPFD("", "gossipcop", "spacy", 'train', ToUndirected())
val_dataset = UPFD("", "gossipcop", "spacy", 'val', ToUndirected())
test_dataset = UPFD("", "gossipcop", "spacy", 'test', ToUndirected())

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/libpyg.so, 0x0006): Symbol not found: __ZN5torch8autograd12VariableInfoC1ERKN2at6TensorE
  Referenced from: <6E2F52AB-0E67-301E-9080-EE4450C3468F> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/libpyg.so
  Expected in:     <DA215AD3-6EAE-3755-B6A5-A8EB4EF952B0> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12

In [2]:
from models import GCNLayer,GCNModel


model=GCNModel(in_channels=300,hidden_dim=120,num_class=1)
criterion=nn.BCEWithLogitsLoss()
optim=torch.optim.Adam(model.parameters(),lr=0.05)
epochs=10
device=torch.device("mps") if torch.mps.is_available() else torch.device("cpu")
print(device)


new_gat,losses,scores,=train_loop(model=model,
                                  loss_fn=criterion,
                                  train_loader=train_loader,
                                  val_loader=val_loader,
                                  max_epochs=epochs,
                                  device=device,
                                  dataset_name="aa",
                                  feature_name="aa")

mps


NotImplementedError: Could not run 'aten::_sparse_coo_tensor_with_dims_and_tensors' with arguments from the 'SparseMPS' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_sparse_coo_tensor_with_dims_and_tensors' is only available for these backends: [MPS, Meta, SparseCPU, SparseMeta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedT